<a href="https://colab.research.google.com/github/parthdhamdhere/NotificationPredictor/blob/main/NotificationPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib
import numpy as np

In [20]:
df = pd.read_csv('fnotifications.csv')  # Replace with your actual filename

print(f"✅ Data loaded successfully: {len(df)} rows")
print(f"\n📊 Dataset columns: {list(df.columns)}")
print(f"\n📈 Label distribution:")
print(df['label'].value_counts())

# Display first few rows
print(f"\n📋 Sample data:")
print(df.head())

# Display app name distribution (optional)
print(f"\n📱 App distribution:")
print(df['app_name'].value_counts().head(10))

# Check for missing values
print(f"\n🔍 Missing values:")
print(df.isnull().sum())


✅ Data loaded successfully: 388 rows

📊 Dataset columns: ['id', 'app_name', 'notification_text', 'label']

📈 Label distribution:
label
URGENT    211
NORMAL    177
Name: count, dtype: int64

📋 Sample data:
   id   app_name                                 notification_text   label
0   1     Zomato  Your order from Pizza Palace is out for delivery  URGENT
1   2  Instagram        anshul_mehta and 12 others liked your post  NORMAL
2   3     Swiggy        Don't miss out! 60% OFF on your next order  NORMAL
3   4   WhatsApp    Mom: Call me when you're free. It's important.  URGENT
4   5      Gmail          Your Amazon.in package has been shipped!  URGENT

📱 App distribution:
app_name
BankApp       28
Google Pay    27
Cred          21
Gmail         21
Shopping      19
Instagram     19
Unknown       18
Swiggy        16
Truecaller    14
Weather       13
Name: count, dtype: int64

🔍 Missing values:
id                   0
app_name             0
notification_text    0
label                0
dtype: i

In [21]:
# Pre-processing: Handle potential missing values
original_size = len(df)

# Remove rows where notification_text or label is missing
df = df.dropna(subset=['notification_text', 'label'])

# Remove empty notification texts
df = df[df['notification_text'].str.strip() != '']

print(f"📊 Data after cleaning: {len(df)} rows ({original_size - len(df)} removed)")

# You can optionally include app_name in features for better predictions
# For now, we'll just use notification_text
X = df['notification_text']
y = df['label']

# Optional: Show some examples by app
print(f"\n📱 Sample notifications by app:")
for app in df['app_name'].unique()[:3]:  # Show first 3 apps
    app_data = df[df['app_name'] == app].head(2)
    print(f"\n{app}:")
    for _, row in app_data.iterrows():
        print(f"  • {row['notification_text']} → {row['label']}")

# Vectorize the Text Data
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X_vectorized = vectorizer.fit_transform(X)

print(f"🔤 Vectorized text shape: {X_vectorized.shape}")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_vectorized, y, test_size=0.2, random_state=42, stratify=y
)

print(f"📚 Training set size: {X_train.shape[0]}")
print(f"🧪 Test set size: {X_test.shape[0]}")

📊 Data after cleaning: 388 rows (0 removed)

📱 Sample notifications by app:

Zomato:
  • Your order from Pizza Palace is out for delivery → URGENT
  • Late night cravings? Order now with 50% OFF → NORMAL

Instagram:
  • anshul_mehta and 12 others liked your post → NORMAL
  • You have a new follower: @techgirl_98 → NORMAL

Swiggy:
  • Don't miss out! 60% OFF on your next order → NORMAL
  • Try new restaurants near you → NORMAL
🔤 Vectorized text shape: (388, 569)
📚 Training set size: 310
🧪 Test set size: 78


In [22]:
# Train the Classification Model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Evaluate the Model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"🎯 Model Accuracy: {accuracy:.2f}")
print(f"\n📋 Classification Report:")
print(classification_report(y_test, y_pred))

# Save the trained model and vectorizer
joblib.dump(model, 'notification_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')
print(f"\n✅ Model and vectorizer saved!")

🎯 Model Accuracy: 0.91

📋 Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.84      1.00      0.91        36
      URGENT       1.00      0.83      0.91        42

    accuracy                           0.91        78
   macro avg       0.92      0.92      0.91        78
weighted avg       0.92      0.91      0.91        78


✅ Model and vectorizer saved!


In [23]:

def predict_priority(notification_text):
    """
    Takes a string of notification text and predicts its priority.
    """
    if not notification_text.strip():
        return None, None

    # Transform the text using the same vectorizer
    text_vectorized = vectorizer.transform([notification_text])

    # Make the prediction
    prediction = model.predict(text_vectorized)
    probability = model.predict_proba(text_vectorized)

    return prediction[0], probability[0]

def display_prediction(text):
    """Display prediction in a nice format"""
    prediction, probability = predict_priority(text)

    if prediction is None:
        print("❌ Error making prediction!")
        return

    # Get confidence score for the predicted class
    confidence = max(probability)

    print("\n" + "="*50)
    print("--- 🔔 NOTIFICATION PREDICTION ---")
    print("="*50)
    print(f"📝 Text: '{text}'")
    print(f"🎯 Predicted: {prediction}")
    print(f"📊 Confidence: {confidence:.2%}")
    print("-"*50)

    if prediction.upper() == 'URGENT':
        print("🚨 Action: Play sound and vibrate! 🚨")
    else:
        print("🤫 Action: Deliver silently. 🤫")
    print("="*50)

print("✅ Prediction function ready!")

✅ Prediction function ready!


In [25]:
# Test with some examples
test_texts = [
    "URGENT ! Need immediate attention!",
    "Weather forecast: Sunny with 75°F temperature",
    "URGENT: Your password expires in 1 hour",
    "New photo uploaded to cloud storage"
]

print("🧪 TESTING PREDICTIONS:")
print("="*60)

for text in test_texts:
    display_prediction(text)


🧪 TESTING PREDICTIONS:

--- 🔔 NOTIFICATION PREDICTION ---
📝 Text: 'URGENT ! Need immediate attention!'
🎯 Predicted: URGENT
📊 Confidence: 57.15%
--------------------------------------------------
🚨 Action: Play sound and vibrate! 🚨

--- 🔔 NOTIFICATION PREDICTION ---
📝 Text: 'Weather forecast: Sunny with 75°F temperature'
🎯 Predicted: NORMAL
📊 Confidence: 66.40%
--------------------------------------------------
🤫 Action: Deliver silently. 🤫

--- 🔔 NOTIFICATION PREDICTION ---
📝 Text: 'URGENT: Your password expires in 1 hour'
🎯 Predicted: URGENT
📊 Confidence: 75.39%
--------------------------------------------------
🚨 Action: Play sound and vibrate! 🚨

--- 🔔 NOTIFICATION PREDICTION ---
📝 Text: 'New photo uploaded to cloud storage'
🎯 Predicted: NORMAL
📊 Confidence: 77.49%
--------------------------------------------------
🤫 Action: Deliver silently. 🤫


In [26]:

# You can test individual predictions here
# Just change the text below and run this cell

test_text = "EMERGENCY: Battery critically low - 2% remaining!"

print("🔍 SINGLE PREDICTION TEST:")
display_prediction(test_text)


🔍 SINGLE PREDICTION TEST:

--- 🔔 NOTIFICATION PREDICTION ---
📝 Text: 'EMERGENCY: Battery critically low - 2% remaining!'
🎯 Predicted: URGENT
📊 Confidence: 79.07%
--------------------------------------------------
🚨 Action: Play sound and vibrate! 🚨


In [17]:
# Interactive testing - run this cell and enter text when prompted
print("🔔 Interactive Notification Predictor")
print("="*40)
print("Enter text below and run the cell to get prediction")

# Get user input in Colab
user_text = input("Enter notification text: ")

if user_text.strip():
    display_prediction(user_text)
else:
    print("❌ Please enter some text!")


🔔 Interactive Notification Predictor
Enter text below and run the cell to get prediction
Enter notification text: family emergency

--- 🔔 NOTIFICATION PREDICTION ---
📝 Text: 'family emergency'
🎯 Predicted: URGENT
📊 Confidence: 60.79%
--------------------------------------------------
🚨 Action: Play sound and vibrate! 🚨
